In [1]:
!git clone --single-branch --branch master https://github.com/DivyaRathod3D/CharNMT.git

Cloning into 'CharNMT'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 135 (delta 27), reused 35 (delta 16), pack-reused 87
Receiving objects: 100% (135/135), 393.87 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Updating files: 100% (53/53), done.


In [2]:
cd CharNMT

/kaggle/working/CharNMT


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class CharDataset(Dataset):
    '''
    Dataset is a iterable that returns input and target sentence. It adds <sos> at the begining, and <eos> at the end, 
    and filling in <pad> if sentence length is less than pre-defined value.
    '''
    
    def __init__(self, x, y, sequence_len, encoder=None):
        # data in the type of pairs of sentence
        data = ''.join(x+y)
        # from collections import Counter
        # vocab_size = 250

        # ct = Counter(data)
        # include = sorted(ct, key=ct.get, reverse=True)
        # if len(include)>vocab_size: include = include[:vocab_size]
        # rule = ''.join(include)
        chars = ['<pad>'] +['<sos>'] + ['<eos>'] + sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)

        print('data has %d characters, %d unique chars, %d sentences.' % (data_size, len(chars), len(x)))
        print('sentence length nine_nine_percentile: %d' % (sequence_len))
        
        self.x, self.y = x, y
        self.ch2i = {ch:i for i,ch in enumerate(chars)}
        self.i2ch = {i:ch for i,ch in enumerate(chars)}
        self.vocab_size = vocab_size
        self.sequence_len = sequence_len
        self.encoder=encoder
    
    def __len__(self):
        return len(self.x) # len x = y
    
    def __getitem__(self, idx):
        
        indx = self.padding([self.ch2i[ch] for ch in self.x[idx]] + [self.ch2i['<eos>']])
        indy = [self.ch2i['<sos>']] + self.padding([self.ch2i[ch] for ch in self.y[idx]] + [self.ch2i['<eos>']])

        x = torch.tensor(indx, dtype=torch.long)
        y = torch.tensor(indy, dtype=torch.long)

        return x,y
                                                                                                                               
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len -1] + [self.ch2i['<eos>']]
                   
        return string

In [5]:
sequence_len = 128
min_len = 1

In [6]:
# process and save data
from utils.pre_processing import *
from utils.utils import *
en, hi = list(), list()
# paths = ["data/pib/","data/hinden/","data/indic/"]
paths = ["data/hinden/"]
for path in paths:
    x = open(path + "Eng.txt", encoding='utf-8').read().split("\n")
    y = open(path + "Hin.txt", encoding='utf-8').read().split("\n")
    x,y = pre_processing(x, y, min_length=min_len, max_length=sequence_len) # remove sentence less than 4 characters
    en += x
    hi += y
    
path = "data/cleaned/"
pickle(path + "en", en)
pickle(path + "hi", hi)
# nine_nine_percentile = int(np.percentile([len(sen) for sen in vi],99))

208155
Some last sentences
To the story | कहानी के लिए
Is something more to the story | कहानी में कुछ और है
The story | कहानी
Something | कुछ


True

In [7]:
# Load saved data
from utils.utils import *
from utils.pre_processing import *

path = "data/cleaned/"
en = pickle(path+"en")
hi = pickle(path+"hi")
en,hi = pre_processing(en, hi, min_length=min_len, max_length=sequence_len) # clip sentences
for i in range(-1,-5,-1):
    print(en[i],'|',hi[i])

175178
Some last sentences
To the story | कहानी के लिए
Is something more to the story | कहानी में कुछ और है
The story | कहानी
Something | कुछ
To the story | कहानी के लिए
Is something more to the story | कहानी में कुछ और है
The story | कहानी
Something | कुछ


In [8]:
dataset = CharDataset(hi, en, sequence_len=sequence_len)

print('sample tensors ', next(iter(dataset)))
print("vocab: ", dataset.ch2i)

data has 14606069 characters, 161 unique chars, 175135 sentences.
sentence length nine_nine_percentile: 128
sample tensors  (tensor([117, 127,  98, 110, 129, 106, 128,  98, 142, 100, 140,  73,   3,  91,
        136,   3, 111, 127, 123,   3,  98, 140,   3,  91, 127, 117, 142, 116,
          3,  91, 117, 110, 127,   3,  96, 127, 124, 128,  85,   6,   3, 120,
        124,   3,  91, 117, 110, 136,   3,  91, 128,   3,  75, 110, 130, 115,
        106, 128,   3, 110, 124, 129,  73,   3, 124, 137,   3,   7,   2,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]), tensor([ 1, 34, 60, 57, 54, 65, 54, 48, 54, 46, 59, 64,  3, 49, 60,  3, 59, 60,
        65,  3, 53, 46, 67, 50,  3, 61, 50, 63, 58, 54, 64, 64, 54, 60, 5

In [9]:
from model.encode_decode_transformer import Transformer, TransformerConfig
from utils.trainer import Trainer, TrainerConfig
tconfig = TrainerConfig(max_epochs=40, batch_size=16, learning_rate=6e-4, grad_norm_clip=1.0, device='cuda',
                       lr_decay=True, warmup_tokens=5000, ckpt_n_print_iter=4000, ckpt_path='checkpoint/transformer_vn_en_char')

mconfig = TransformerConfig(vocab_size=dataset.vocab_size, sequence_len=dataset.sequence_len, embed_dim=256,
                           n_block=8, n_head=8, device=tconfig.device)

In [10]:
model = Transformer(mconfig)

In [11]:
sentences = ["मैं उससे मिलना चाहूँगा।",
           "यह वही चीज़ है जिसकी मुझे ज़रूरत है।",
           "मेरी चिंता मत करो।",
           "उसने मुझे दो किताबें उधार दीं।",
           "मैं इस योजना से सहमत हूँ।"
            ]
trainer = Trainer(model, dataset, tconfig, test_dataset=sentences, collate=None)

In [12]:
# load pre-trained weights
# from utils.utils import pickle
# model.load_state_dict(pickle(tconfig.ckpt_path)) # load


In [13]:
trainer.train()

epoch: 1 | train loss: 900.52105  | lr: 1.920000e-06:   0%|          | 0/10945 [00:00<?, ?it/s]

[]


epoch: 1 | train loss: 1.68467  | lr: 4.387871e-04:  37%|███▋      | 4000/10945 [06:20<10:18, 11.24it/s]  

['And this is this is this is this is that their country.', 'This is this is this is this is this is this is.', 'This is their changer of India.', 'This is this is that this is the partion of India.', 'This is this is this is this is this is this is.']


epoch: 1 | train loss: 1.48238  | lr: 1.065213e-04:  73%|███████▎  | 8000/10945 [12:38<04:24, 11.15it/s]  

...

epoch: 40 | train loss: 0.42355  | lr: 3.062709e-04:  37%|███▋      | 4002/10945 [06:12<1:28:06,  1.31it/s]

["I'd like to meet it.", 'This is the same thing I need to.', "Don't worry my concern.", 'He gave me two books.', 'I agree with this plan.']


epoch: 40 | train loss: 0.44007  | lr: 6.000000e-05:  73%|███████▎  | 8002/10945 [12:20<39:02,  1.26it/s]  

["I'd like to meet him.", "It's the same thing I need to.", "Don't worry my concern.", 'He gave me two books.', 'I agree with this plan.']


epoch: 40 | train loss: 0.42123  | lr: 6.000000e-05: 100%|██████████| 10945/10945 [16:47<00:00, 10.87it/s]


In [14]:
samples = ["मेरे ख़याल से तुम सही हो।",
           "मैं नौ बजे से पहले वापस आ जाऊँगा।",
           "ये किताबें मेरीं हैं।",
           "तुम मंगलवार को खाली हो क्या?",
           "वह एक राजा से बहुत ज़्यादा था।"
          ]
result = model.generate_output(samples, dataset, top_k=5, print_process=True)
print(result)

100%|██████████| 5/5 [00:04<00:00,  1.03it/s]

["I think you're right.", "I'm going to go back to the nine stuff.", 'These books are my books.', ' what are you the Mangalwar? ', 'He was very much more than a King.']


In [15]:
# benchmarking using bleu score
path = "data/hinden/"
en = open(path+"test.en.txt", encoding='utf-8').read().split("\n")
hi = open(path+"test.hi.txt", encoding='utf-8').read().split("\n")
en, hi = pre_processing(en, hi, min_length=min_len, max_length=sequence_len) # remove sentence less than 4 characters

result = model.generate_output(hi, dataset, top_k=5, print_process=True)

2508
Some last sentences
32 chartered planes have been booked to ferry the guests to and fro. | मेहमानों को लानेले जाने के लिए 32 चार्टर्ड विमानों की व्यवस्था की गई है।
250 VIPs have been invited to this royal party. | इस शाही पार्टी के लिए 250 वीवीआईपी लोगों को भी आमंत्रित किया गया है।
It is noteworthy that both Nita and Isha are professional, classical dancers. | गौरतलब है कि नीता और ईशा दोनों ही प्रोफेशनल क्लासिकल डांसर है।
It is being said that Nita Ambani and her daughter Isha may also perform in this. | बताया जा रहा है कि नीता अंबानी और उनकी बेटी ईशा भी इसमें अपनी प्रस्तुती दे सकती है।


100%|██████████| 1495/1495 [25:23<00:00,  1.02s/it]


In [16]:
from utils.utils import *
score, references, candidates = bleu_score(en, result)
print(score)

6.617218344318052
